<center> <h2> DS 3000 - Fall 2020</h2> </center>
<center> <h3> DS Report </h3> </center>


<center> <h3> Project Title</h3> </center>
<center><h4>Team Member Names</h4></center>


<hr style="height:2px; border:none; color:black; background-color:black;">

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text


#### Executive Summary:

Add your summary here (100-150 words)

Provide a brief summary of your project. After reading this executive summary, your readers should have a rough understanding of what you did in this project. You can think of this summary in terms of the four sections of the report and write 1-2 sentences describing each section.



<hr style="height:2px; border:none; color:black; background-color:black;">

## Outline
1. <a href='#1'>INTRODUCTION</a>
2. <a href='#2'>METHOD</a>
3. <a href='#3'>RESULTS</a>
4. <a href='#4'>DISCUSSION</a>

<a id="1"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

## 1. INTRODUCTION

In this section, orient your readers to your project. You've already written some of these in previous deliverables. Based on your final analysis, revise your problem statement and write a concise introduction section. This section should touch upon the following points, but should be written in full paragraphs. Your writing should incorporate all of these points (and more if you like) in a coherent way. Remember that you are trying to convince your readers that this is an important problem to tackle. 

Problem Statement
* Describe the problem you would like to tackle. 
* What is the topic of your project? 
* What do you want to learn about it?

Significance of the Problem
* Why is it important to tackle this problem in your project?
* In what ways could the insights from this project be useful?
* **(optional)** Has there been previous work on your topic in terms of applying ML techniques to analyze similar datasets? Do some research into your topic. What algorithms were used? What was the performance of those algorithms? Cite your sources appropriately. You can use the numbered reference format or APA (if you are more comfortable with it).

Questions/Hypothesis
* End this section with a list of questions and hypotheses
* You should tie these questions/hypotheses to the problem statement and its significance
    * e.g. Given the aforementioned problem and its importance, we set out to tackle the following questions:
    
**Requirement:**
* You should have at least one question tapping into the comparison of various machine learning algorithms (at least three) in predicting/classifying your target variable from your features variables.
* You should have at least one hypothesis regarding the relationship between two variables, which may be different from your ML problem.

While this section provides bullet points as prompts, your Introduction should consist of coherent paragraphs. As noted in the rubric, you are required delete all the prompts from this template, except for headings. Failing to do so will result in substantial point deductions.

**[IN PROGRESS]**

For our final project, we plan to analyze patterns in presidential speech in order to understand how the role of the president in establishing the national social agenda has changed over time. In particular, we’d like to focus on tracking the prevalence of different themes over time using feature analysis. Using our final output dataset, we will be able to visualize the progression and popularity of different social issues in presidential rhetoric at different points in time. For example, we plan to look into such patterns as gender-specific pronouns, and analyze speech readability using the Flesh-Kincaid test. Our finite dataset has 997 rows, restricted by the number of presidential speeches; this data was acquired primarily using an online corpora, and we used web-scraping to acquire the speeches of the 45th president. We have multiple features in the dataset -- for readability, we consider such attributes as word count, sentence count, and average syllables per word, and for sentimentality, we consider the frequencies of gendered pronouns, and terms pertaining to race/religion. The target/outcome variables for our dataset include political party, president, and time period, the last of which is to be indexed using 20-year blocks. We plan to use this data primarily to visualize trends of progressivism in presidential speeches over time; however, we also plan to work towards training a model to predict these target variables.

Questions regarding our dataset:

* Are there significant presidential speech sentimentality trends over time?
* Which visualization types fit best for representing the attributes and trends of our dataset?
* What machine learning algorithm (out of the ones we’ve seen used in class) can best classify the presidential speeches?
* To what degree of accuracy can we get a machine learning algorithm to predict the aforementioned target variables? (Using cross-validation procedures)
* Which feature variable will play the largest role in classification?
* Can we train a machine learning classification algorithm to classify the political party from which the speeches came? 
* (This will utilize only speech data from after 1936, when the Democratic and Republican parties established their modern platforms.)
* We also plan to test multiple hypotheses, including:
* The frequency of female-gendered terms will increase over time.
* The frequency of male-gendered terms will decrease over time. 
* The use of religious language will decrease over time. 
* The use of equality-related rhetoric will decrease over time. 
* Religious language will be a key indicator of political party.


Our project primarily tackles a classification problem, as we plan to train our model to predict political party (binary classification), speaker (multiclass classification), and year (multiclass classification using feature discretization, nbins= 20). We also plan to experiment with regressive prediction, utilizing speech year as a continuous outcome variable. Algorithm-wise, we plan to experiment with each of the models covered in lecture, as the sklearn library makes it easy to compare the performance of multiple algorithms on a dataset (kNN, SVM, Naïve-Bayes, Decision Trees; Linear, Polynomial, Ridge, and Lasso regression). We are particularly interested in evaluating the importance of each feature, using univariate and model-based feature selection. As mentioned, we plan to try using feature discretization for the regressive prediction component; we also plan to experiment using both bag-of-words and n-grams for feature extraction. By experimenting with preliminary correlation visualizations, we will be able to gauge whether our features are likely to demonstrate predictive capabilities; either way, we may chose to expand our feature analysis into other linguistic components -- for example, it may be an interesting challenge to determine if there are certain words which maximize prediction accuracy of the model, or consider such features as speech location at different geographical scales.


<a id="2"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

## 2. METHOD

### 2.1. Data Acquisition

* Describe where you obtained your data. Provide a link to the original source. 
* If you scraped your data, include your code as a **separate** script file.
* Your data should be stored in an online repository (e.g., GitHub) and your code should retrieve your data from that online resource. You can read csv files from the Web in the same way that you read files from local drive.
* Describe the dataset (i.e., what it is about) and the number of variables/rows included.


In [2]:
url = "https://raw.githubusercontent.com/odaily/DS3000-Project/master/final_proj_dataset3.csv"
df = pd.read_csv('final_proj_dataset.csv', index_col=0)#url, index_col=0)

### 2.2. Data Analysis
* For your hypotheses, what are your IVs and DVs?
* Specifically describe your predictive model. What outcome variable are you going to predict from what feature variables? Why do you think those are important predictors?
* Describe why this is a supervised ML problem and identify the sub-category of the learning task (e.g. classification).
* What machine learning algorithms are you going to use? **Why?** You should compare at least three algorithms.

<a id="3"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

## 3. RESULTS

### 3.1. Data Wrangling
In this section, you should do the following and explain why you are doing what you are doing. For each, you should include your code in a cell, followed by a sample output. For instance, if you are one-hot encoding one of your variables, you should first describe what it is and why you are doing it. You should then include your code in a cell, and the sample output should be available as well.

* Perform simple data cleaning (delete extra columns, deal with NA values, etc.)
* Perform data wrangling to get your features and target values (e.g., grouping your dataframe by columns, applying functions to format dataframes, etc.)
* Preprocess your variables (e.g., scaling/transforming feature variables to normalize them)
* Perform feature extraction (dummy variables, new features from existing features, etc.)
* Use one feature selection technique to select a subset of your original features


In [22]:
# CLEANING 
# Some of our data has incorrect or missing values for political party, presidents should keep their same political affiliation
# as they had the year they first entered office, we will use data about when they entered office, find their political
# party for that year, and apply that to all rows for each president.
# Also, there are NaN values for time period for the speeches made by Donald Trump, FDR, and Ronald Reagan.
# This is because the years for these rows are truncated and are missing a year

In [23]:
def getFirstYear(presName):
    return yearsAndNames[yearsAndNames['name'] == presName]['first year in office'].item()
def getPartyIndex(partyRaw, year):
    if partyRaw == "Independent":
        return 0
    elif partyRaw == "Federalist":
        return 1
    elif partyRaw == "Democratic-Republican" or partyRaw == "Democratic-Republican/National Republican":
        return 2
    elif partyRaw == "Whig":
        return 3
    elif partyRaw == "Republican" and year < 1865:
        return 4
    elif partyRaw == "Democratic" and year < 1865:
        return 5
    elif partyRaw == "Republican/National Union" or partyRaw == "Democratic/National Union":
        return 6
    elif partyRaw == "Democratic" and year >= 1865 and year < 1961:
        return 7
    elif partyRaw == "Republican" and year >= 1865 and year < 1961:
        return 8
    elif partyRaw == "Democratic" and year >= 1961:
        return 9
    elif partyRaw == "Republican" and year >= 1961:
        return 10
    

In [24]:
# the data to get the year they took office
yearsUrl = "https://gist.githubusercontent.com/namuol/2657233/raw/74135b2637e624848c163759be9cd14ae33f5153/presidents.csv"
yearsDf = pd.read_csv(yearsUrl)
yearsAndNames = pd.DataFrame(columns = ["name", "first year in office"])
yearsAndNames["first year in office"] = yearsDf['Took office '].str[-4:]
namesRaw = yearsDf["President "].str.split().str[-1].str.lower()
yearsAndNames["name"] = namesRaw

# changing certain names to match our naming conventions
yearsAndNames.iloc[7] = ['vanburen', 1837]
yearsAndNames.iloc[22] = ['bharrison', 1889]
yearsAndNames.iloc[22] = ['bharrison', 1889]
yearsAndNames.iloc[22] = ['bharrison', 1889]
yearsAndNames.iloc[31] = ['fdroosevelt', 1933]
yearsAndNames.iloc[42] = ['gwbush', 2001]
yearsAndNames.iloc[5] = ['jqadams', 1825]
yearsAndNames.iloc[35] = ['lbjohnson', 1963]
yearsAndNames.loc[44]= ['trump', 2017]


parties = []
yearsAndNames['party'] = yearsDf['Party ']

#adding in trump's party
yearsAndNames.loc[44]= ['trump', 2017,'Republican']

# parsing party and changing it to be an index 
for name in yearsAndNames['name']:
    first_year = int(getFirstYear(name))
    party = yearsAndNames[yearsAndNames['name']==name].party.item().strip()
    parties.append(getPartyIndex(party, first_year))
# adds the parties to the df
yearsAndNames['party'] = parties

# sets all president's political party to be the one it was when they entered the office
for name in yearsAndNames['name']:
    first_year = int(getFirstYear(name))
    df.loc[df.president == name, 'political_party'] = yearsAndNames[yearsAndNames['name']==name].party.item()

# fixing NaN time period values

# getting correct time period for trump from line 898
trump_time_period = df.iloc[898]['time_period']
# setting all of trump's data to match this
df.loc[df.president == 'trump', 'time_period'] = trump_time_period


# fixing the truncated years for two presidents
df.at[208,'year'] = 1940
df.at[814,'year'] = 1987

#checking for null values in the df
#df.isnull().sum()

# Our data is now clean!yh

In [25]:
# library of gendered terms
femaleGendered = ["she", "her", "hers", "woman", "women", "girl", "girls", "female"]
maleGendered = ["he", "him", "his", "man", "men", "boy", "boys", "male"]
equality = ["equality", "equal", "fairness", "equal rights", "equal opportunities", "egalitarian", "egalitarianism", "equity",
           "equitability"]
race = ["white", "black", "asian", "indian", "african", "caucasian", "ethnicity", "european", "hispanic", "indigenous", 
       "racism", "minority", "race", "predjudice", "racial", "latin", "latino", "latina", "jewish", "jew", "african american",
        "african-american",
       "whites", "blacks", "people of color", "native american", "native"]
religion = ["religion", "jewish", "christian", "hindu", "judaism", "hinduism", "christianity", "lutherian", "catholic", 
           "catholocism", "protestant", "protestantism", "god", "buddist", "buddism", "religous"]

In [26]:
# function that sums the values of the given terms in the given dictionary
def sumTermValues(terms, dictionary):
    sum = 0
    for term in terms:
        if dictionary.get(term) is not None:
            sum = sum + dictionary.get(term)
    return sum

In [27]:
def getImportanceValue(president, terms):
    speeches = []
    
    for speech in df[df['president'] == president]['speech']:
        speeches.append(speech.lower())
    # cannot use english stop words because it will ignore gendered terms like he/she
    vectorizer = TfidfVectorizer(token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b')
    vect = vectorizer.fit(speeches)
    tfidf = vectorizer.fit_transform(speeches)
    values = tfidf.toarray().tolist()
    flatList = []
    for elem in values:
        flatList.extend(elem)
    keys = vectorizer.get_feature_names()
    dictionary = {} 
    for key in keys: 
        for value in flatList: 
            dictionary[key] = value 
            flatList.remove(value) 
            break 
    return sumTermValues(terms, dictionary)

In [28]:
def label_president(row, terms):
    return getImportanceValue(row['president'], terms)

In [29]:
# FEATURE EXTRACTION -- also my intial feature extraction counts move that here
# The feature extraction we will perform includes getting the weight of certain words in the corpus.
speeches = []
for speech in df['speech']:
    speeches.append(speech.lower())
# cannot use english stop words because it will ignore gendered terms like he/she
vectorizer = TfidfVectorizer(token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', min_df=20)
vect = vectorizer.fit(speeches)
tfidf = vectorizer.fit_transform(speeches)
values = tfidf.toarray().tolist()
flatList = []
for elem in values:
    flatList.extend(elem)
keys = vectorizer.get_feature_names()
dictionary = {} 
for key in keys: 
    for value in flatList: 
        dictionary[key] = value 
        flatList.remove(value) 
        break 

# doing this for the entire corpus, should I do it per president or per time period -- I think per president 
# then the presidents of a certain time period will help the classification by time period
print("female gendered: ", sumTermValues(femaleGendered, dictionary))
print("male gendered: ", sumTermValues(maleGendered, dictionary))
print("terms for equality: ", sumTermValues(equality, dictionary))
print("terms for race: ", sumTermValues(race, dictionary))
print("terms for religion: ", sumTermValues(religion, dictionary))


female gendered:  0.0
male gendered:  0.09797811787081741
terms for equality:  0.014049833538115326
terms for race:  0.0
terms for religion:  0.004701899831922961


In [30]:
#FEATURE SELECTION
#print("trump terms for men: ", getImportanceValue('trump', equality))
df['importance_of_equality'] = df.apply (lambda row: label_president(row, equality), axis=1)
df['importance_of_race'] = df.apply (lambda row: label_president(row, race), axis=1)
df['importance_of_religion'] = df.apply (lambda row: label_president(row, religion), axis=1)
df['importance_of_female_gendered_terms'] = df.apply (lambda row: label_president(row, femaleGendered), axis=1)
df['importance_of_male_gendered_terms'] = df.apply (lambda row: label_president(row, maleGendered), axis=1)


In [31]:
df

,speech,word_count,sentence_count,average_words,average_syl_per_word,flesch_kincaid_score,flesch_kincaid_grade_level,total_gendered_terms,female_gendered_terms,male_gendered_terms,...,terms_for_religion,political_party,president,time_period,year,importance_of_equality,importance_of_race,importance_of_religion,importance_of_female_gendered_terms,importance_of_male_gendered_terms
0,The personal inconveniences to the members of ...,3034,79,39.379747,1.708306,22.341881,19.926110,28,0,28,...,0,1.0,adams,0.0,NaN,0.011209,0.0,0.004739,0.000000,0.101894
1,"When it was first perceived, in early times, t...",2322,34,68.823529,1.690353,-6.024758,31.197344,14,0,14,...,3,1.0,adams,0.0,NaN,0.011209,0.0,0.004739,0.000000,0.101894
2,Gentlemen of the Senate and Gentlemen of the H...,2224,59,38.661017,1.681205,25.364122,19.326016,12,5,7,...,1,1.0,adams,0.0,NaN,0.011209,0.0,0.004739,0.000000,0.101894
3,As the safety and prosperity of nations ultima...,663,9,73.555556,1.731523,-14.310767,33.528643,5,0,5,...,3,1.0,adams,0.0,NaN,0.011209,0.0,0.004739,0.000000,0.101894
4,It is with peculiar satisfaction that I meet t...,1500,37,41.459459,1.730000,18.395649,20.993189,7,0,7,...,0,1.0,adams,0.0,NaN,0.011209,0.0,0.004739,0.000000,0.101894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,"Gentlemen of the Congress:\n\nOnce more, as re...",2502,89,28.808989,1.644684,38.453589,15.052780,27,20,7,...,0,7.0,wilson,6.0,NaN,0.010702,0.0,0.009363,0.040848,0.045702
993,GENTLEMEN OF THE CONGRESS: \n\nThe year that h...,5468,151,36.993377,1.550293,38.131967,17.130870,35,1,34,...,2,7.0,wilson,6.0,NaN,0.010702,0.0,0.009363,0.040848,0.045702
994,GENTLEMEN OF THE CONGRESS: \n\nEight months ha...,3921,137,29.321168,1.520786,48.415560,13.790525,24,11,13,...,2,7.0,wilson,6.0,NaN,0.010702,0.0,0.009363,0.040848,0.045702
995,TO THE SENATE AND HOUSE OF REPRESENTATIVES:\n\...,4761,160,30.550000,1.661416,35.270984,15.929205,38,13,25,...,0,7.0,wilson,6.0,NaN,0.010702,0.0,0.009363,0.040848,0.045702


In [32]:
df.to_csv('final_proj_dataset_with_importance.csv')

### 3.2. Data Exploration
* Generate appropriate data visualizations for your key variables identified in the previous section
* You should have at least three visualizations (and at least two different visualization types)
* For each visualization provide an explanation regarding the variables involved and an interpretation of the graph.
* If you are using Plotly, insert your visualizations as images as well (upload the graph images to an online source, e.g. github, and embed those into the cells in Jupyter Notebook). This is a requirement.


### 3.3. Model Construction
* Conduct your hypothesis test(s) here.
* For your machine learning question(s), use the Training, Validation, and Testing approach through GridSearch
* Apply machine learning algorithms (apply at least three different algorithms)
* Train your algorithms

In [39]:
features = df.drop(['political_party','president','time_period', 'speech', 'year'], axis=1)
target_party = df['political_party']
target_pres = df['president']
target_period = df['time_period']
all_targets = {'Political Party':target_party, 'President':target_pres, 'Time Period':target_period}

from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

estimators = {
    'k-Nearest Neighbor': KNeighborsClassifier(),
    'Gaussian Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier()
}

### 3.4. Model Evaluation
* Evaluate the performance of your algorithms on appropriate evaluation metrics, using your **validation set**
    * Use at least two different metrics 
* Evaluate your results from multiple ML models and hypothesis tests
    * What was the performance of each algorithm in plain English? Is there any indication of overfitting/underfitting?
    * Was there a significant difference? Use the template from lecture slides when reporting the results of your hypothesis testing.

In [41]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

for target_name, target in all_targets.items():
    print(f"{target_name} models:")
    for estimator_name, estimator in estimators.items():
        kfold = KFold(n_splits=10, shuffle=True, random_state=820)
        scores = cross_val_score(estimator=estimator, X=features, y=target, cv=kfold)
        
        print(f"{estimator_name}:\n\tMean accuracy: {scores.mean():.2%}, std: {scores.std():.2%}")
    print('\n')
print("Done!")        

Political Party models:
k-Nearest Neighbor:
	Mean accuracy: nan%, std: nan%
Gaussian Naive Bayes:
	Mean accuracy: nan%, std: nan%
Decision Tree:
	Mean accuracy: nan%, std: nan%


President models:
k-Nearest Neighbor:
	Mean accuracy: nan%, std: nan%


C:\Users\losts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\neighbors\_base.py", line 1131, in fit
    X, y = self._validate_data(X, y, accept_sparse="csr",
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 73, in inner_f
    return f(**kwargs)
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 796, in check_X_y
    X = check_array(X, accept_sparse=accept_sparse,
  

C:\Users\losts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\naive_bayes.py", line 210, in fit
    X, y = self._validate_data(X, y)
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 73, in inner_f
    return f(**kwargs)
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 796, in check_X_y
    X = check_array(X, accept_sparse=accept_sparse,
  File "C:\Users\losts\anaco

Gaussian Naive Bayes:
	Mean accuracy: nan%, std: nan%
Decision Tree:
	Mean accuracy: nan%, std: nan%


Time Period models:
k-Nearest Neighbor:
	Mean accuracy: nan%, std: nan%
Gaussian Naive Bayes:
	Mean accuracy: nan%, std: nan%
Decision Tree:
	Mean accuracy: nan%, std: nan%


Done!


C:\Users\losts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 890, in fit
    super().fit(
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 156, in fit
    X, y = self._validate_data(X, y,
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\base.py", line 429, in _validate_data
    X = check_array(X, **check_X_params)
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 73, in inner_f
    return f(**kwargs)
  File "C:\Users\losts\anaconda3\lib\site-packages\sklearn\utils\validat

### 3.5. Model Optimization
* Tune your models using appropriate hyperparameters
* Explain why you are doing this (e.g., to avoid overfitting, etc.)

### 3.6. Model Testing
* Test your tuned algorithms using your **testing set**

<a id="4"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

## 4. DISCUSSION
* Interpret your findings from 3.4., 3.5, and 3.6
    * Which algorithms did you compare?
    * Which algorithm(s) revealed best performance?
    * Which algorithm(s) should be used for your predictive model?
    * Based on your findings, can we use the features in your dataset to predict the outcome variable you identified using the algorithms you've applied? (It is okay if the answer is no. We're interested in the process, not the performance of the model.)
* For your hypotheses, interpret the results. What does it mean to have significant/non-significant differences with regards to your data?


* End this section with a conclusion paragraph containing some pointers for future work
    * (e.g., get more data/features, perform another analysis, etc.)

<a id="5"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

### CONTRIBUTIONS
* Describe each team member's contributions to the report (who did what in each section)
* Remember this is a team effort!
* Each member of your team will provide peer evaluation of other team members. Your final grade on the project will be based on those peer evaluations. A survey will be shared after the deadline for this deliverable.